# Preprocessing

## 0) Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/wm-project/"
%cd /content/drive/My\ Drive/wm-project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
 02062021MusicRecommenderTeam3.ipynb
 26052021MusicRecommenderTeam3.ipynb
 all_predictions_scores.csv
 als_predictions_scores1.csv
 als_predictions_scores2.csv
 als_predictions_scores.csv
 back22052021MusicRecommenderTeam3.ipynb
 df_artist.csv
 df_general.csv
 df_outlierDtected.csv
 df_preprocessed.csv
 df_test.csv
 df_train_cleaned.csv
 df_train_to_explore.csv
 lookup_artist_names.csv
 MusicRecommender_01_Preprocessing_backup.ipynb
 MusicRecommender_01_Preprocessing.ipynb
 MusicRecommender_02_Modelling.ipynb
 nrartist_df.csv
 parameter_tuning
 rec_train.csv
 tasks.gsheet
'Team 3 - Music recommender.gslides'
 userid-profile.tsv
 userid-timestamp-artid-artname-traid-traname.tsv
 WM_Slides_08062021.pptx
/content/drive/.shortcut-targets-by-id/16Jxh

In [ ]:
# import dataset and drop time and track columns
import pandas as pd 

df_original = pd.read_csv(
    'userid-timestamp-artid-artname-traid-traname.tsv',
    sep='\t',
    error_bad_lines=False,
    names=['user_id', 'timestamp', 'artist_id',
           'artist_name', 'track_id', 'track_name'],
    header=None,
    # remove malicious rows
    skiprows=[
        2120260-1, 2446318-1, 11141081-1,
        11152099-1, 11152402-1, 11882087-1,
        12902539-1, 12935044-1, 17589539-1
    ]
)
df_original.drop(axis=1,
                 columns=['timestamp', 'track_id', 'track_name'],
                 inplace=True)
df_original.head(3)

,user_id,artist_id,artist_name
0,user_000001,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish
1,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一
2,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一


In [ ]:
# function for status of dataframe
def pruning_status (df_old, df_new):
  # group dataset if not done
  df_old_grouped = df_old
  if 'absolute' not in df_old.columns:
    df_old_grouped = df_old.copy()
    df_old_grouped['absolute'] = df_old_grouped.groupby(['user_id', 'artist_id']).user_id.transform('count')
    df_old_grouped.drop_duplicates(keep='first', inplace=True)

  df_new_grouped = df_new
  if 'absolute' not in df_new.columns:
    df_new_grouped = df_new.copy()
    df_new_grouped['absolute'] = df_new_grouped.groupby(['user_id', 'artist_id']).user_id.transform('count')
    df_new_grouped.drop_duplicates(keep='first', inplace=True)

  data = [["Entries", 
           len(df_old.index), 
           len(df_new.index)], 
          ["Artists", 
            df_old.artist_id.unique().shape[0], 
            df_new.artist_id.unique().shape[0]],
          ["Users", 
           df_old.user_id.unique().shape[0], 
           df_new.user_id.unique().shape[0]],
          ["Density", 
           round(len(df_old_grouped.index) / (df_old.user_id.unique().shape[0] * df_old.artist_id.unique().shape[0]), 5), 
           round(len(df_new_grouped.index) / (df_new.user_id.unique().shape[0] * df_new.artist_id.unique().shape[0]), 5)]]
  stats = pd.DataFrame(data, columns=["Data", "Old", "New"])
  stats["Kept in %"] = stats['New'] / stats['Old'] * 100
  return stats

## 1) Remove Null Values

In [ ]:
#  Remove null values

print(df_original.isnull().sum())

df_cleaned = df_original.copy()
df_cleaned.dropna(inplace=True)

print(df_cleaned.isnull().sum())

user_id             0
artist_id      600848
artist_name         0
dtype: int64
user_id        0
artist_id      0
artist_name    0
dtype: int64


In [ ]:
pruning_status(df_original, df_cleaned)

,Data,Old,New,Kept in %
0,Entries,1.909885e+07,1.849800e+07,96.854010
1,Artists,1.072960e+05,1.072950e+05,99.999068
2,Users,9.920000e+02,9.920000e+02,100.000000
3,Density,8.490000e-03,7.700000e-03,90.694935


## 2) Group and Count

In [ ]:
# Group to artist-user touples and count occurances

df_grouped = df_cleaned.copy()

df_grouped['absolute'] = df_grouped.groupby(['user_id', 'artist_id']).user_id.transform('count')
df_grouped.drop_duplicates(keep='first', inplace=True)
df_grouped.head()

,user_id,artist_id,artist_name,absolute
0,user_000001,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,1
1,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,855
14,user_000001,ba2f4f3b-0293-4bc8-bb94-2f73b5207343,Underworld,779
16,user_000001,a16e47f5-aa54-47fe-87e4-bb8af91a9fdd,Ennio Morricone,12
17,user_000001,463a94f1-2713-40b1-9c88-dcc9c0170cae,Minus 8,18


In [ ]:
pruning_status(df_cleaned, df_grouped)

,Data,Old,New,Kept in %
0,Entries,1.849800e+07,819785.0000,4.431748
1,Artists,1.072950e+05,107295.0000,100.000000
2,Users,9.920000e+02,992.0000,100.000000
3,Density,7.700000e-03,0.0077,100.000000


## 2) Train-test  split

In [ ]:
# Only keep artists that are listened by at least two users, that all artists can ocure in both, train and test set
df_final = df_grouped.groupby('artist_id').filter(lambda x: len(x) >1)
pruning_status(df_grouped, df_final)

,Data,Old,New,Kept in %
0,Entries,819785.0000,771303.00000,94.086010
1,Artists,107295.0000,58813.00000,54.814297
2,Users,992.0000,992.00000,100.000000
3,Density,0.0077,0.01322,171.688312


In [ ]:
# Export lookup_artist_names table
lookup_artist_names = df_final[['artist_id', 'artist_name']]
lookup_artist_names.drop_duplicates(keep='first', inplace=True)
lookup_artist_names.to_csv('lookup_artist_names.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Drop column artist_name for train/test split
df_final = df_final.drop(columns=['artist_name'])

In [ ]:
# Split in 75% train and 25% test set, stratified by artist_id
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_final,
                                     test_size=0.25,
                                     random_state=41,
                                     stratify=df_final[['artist_id']])

In [ ]:
# export test set
df_test.to_csv('df_test.csv', index=False)

In [ ]:
pruning_status(df_final, df_test)

,Data,Old,New,Kept in %
0,Entries,771303.00000,192826.00000,25.000032
1,Artists,58813.00000,44711.00000,76.022308
2,Users,992.00000,990.00000,99.798387
3,Density,0.01322,0.00436,32.980333


In [ ]:
pruning_status(df_final, df_train)

,Data,Old,New,Kept in %
0,Entries,771303.00000,578477.00000,74.999968
1,Artists,58813.00000,58813.00000,100.000000
2,Users,992.00000,992.00000,100.000000
3,Density,0.01322,0.00992,75.037821


# Train set cleaning

## 3) Remove Outliers

In [ ]:
# Calculate percentage of listening

df_train_1 = df_train.copy()

df_train_1['user_total'] = df_train_1.groupby('user_id').absolute.transform('sum')
df_train_1['percent'] = df_train_1['absolute'] / df_train_1['user_total'] * 100

df_train_1.drop(columns=['user_total'], inplace=True)
df_train_1.head(4)

,user_id,artist_id,absolute,percent
14500789,user_000774,7acc6c97-2654-4ffa-945f-daeabaa8e5ae,19,0.034935
18074101,user_000946,0908d42b-6f56-44cf-a992-5004be8323bb,1,0.017215
14416422,user_000770,65314b12-0e08-43fa-ba33-baaa7b874c15,127,0.716340
4077647,user_000206,4213b9bb-9705-49dd-bd73-0ae7982bf4ca,1,0.003837


In [ ]:
df_train_1.describe()

,absolute,percent
count,578477.000000,578477.000000
mean,23.470090,0.171485
std,127.988193,0.764555
min,1.000000,0.000750
25%,1.000000,0.008548
50%,3.000000,0.025793
75%,11.000000,0.097245
max,26496.000000,100.000000


In [ ]:
# Remove songs, where the max percentage is below 0.5%

df_train_1['artist_max'] = df_train_1.groupby('artist_id').percent.transform('max')
df_train_1 = df_train_1[(df_train_1['artist_max'] > 0.5)]

pruning_status(df_train, df_train_1)

,Data,Old,New,Kept in %
0,Entries,578477.00000,363496.00000,62.836725
1,Artists,58813.00000,10335.00000,17.572646
2,Users,992.00000,992.00000,100.000000
3,Density,0.00992,0.03545,357.358871


## 4) Remove Noise
Single listening can be noise or dislike

In [ ]:
# Remove noise: Remove once listened artists, when percentage shows also irrelevance (<0.5%)
df_train_2 = df_train_1[(df_train_1['percent'] > 0.5) | (df_train_1['absolute'] > 2)]
pruning_status(df_train_1, df_train_2)

,Data,Old,New,Kept in %
0,Entries,363496.00000,216941.00000,59.681812
1,Artists,10335.00000,10335.00000,100.000000
2,Users,992.00000,992.00000,100.000000
3,Density,0.03545,0.02116,59.689704


In [ ]:
df_train_2.drop(columns=['percent', 'artist_max'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## 5) Recalculate relative listenings

In [ ]:
import numpy as np

def apply_scales(df):
    # (Re-)Calculate relative listening: Divide absolute listening by total listenings by user.
    df['user_total'] = df.groupby('user_id').absolute.transform('sum')
    df['relative'] = df['absolute'] / df['user_total']
    
    return df

In [ ]:
df_train_3 = df_train_2.copy()
apply_scales(df_train_3).head(3)

,user_id,artist_id,absolute,user_total,relative
14500789,user_000774,7acc6c97-2654-4ffa-945f-daeabaa8e5ae,19,38970,0.000488
14416422,user_000770,65314b12-0e08-43fa-ba33-baaa7b874c15,127,17154,0.007404
11956022,user_000646,3e30aebd-0557-4cfd-8fb9-3945afa5d72b,8,1211,0.006606


In [ ]:
df_train_3.describe()

,absolute,user_total,relative
count,216941.000000,216941.000000,216941.000000
mean,54.583928,19825.516873,0.004573
std,204.457480,19439.663010,0.013048
min,1.000000,1.000000,0.000023
25%,5.000000,6423.000000,0.000416
50%,12.000000,13384.000000,0.001223
75%,39.000000,26583.000000,0.004087
max,26496.000000,129789.000000,1.000000


## 6) Export train set

In [ ]:
# See train set cleaning
pruning_status(df_train, df_train_3)

,Data,Old,New,Kept in %
0,Entries,578477.00000,216941.00000,37.502096
1,Artists,58813.00000,10335.00000,17.572646
2,Users,992.00000,992.00000,100.000000
3,Density,0.00992,0.02116,213.306452


In [ ]:
# export df_preprocessed
df_train_3.to_csv('df_train_cleaned.csv', index=False)